In [1]:


import numpy as np
import tensorflow as tf

from drowsiness_detection import config
from drowsiness_detection.models import build_dummy_tf_classifier, ThreeDStandardScaler,build_dense_model, build_lstm_model, build_cnn_model, step_decay
import tensorflow.keras as keras


In [2]:
window_size = 10
recording_frequency = 30
exclude_by = "a"

config.set_paths(recording_frequency, window_size)
config.PATHS.WINDOW_DATA

PosixPath('/home/tim/Windows/Windows_30_Hz/WindowData/10_sec')

In [3]:
num_samples = 20
X = np.random.random(num_samples * 300 * 7).reshape((num_samples, 300, 7))
y = np.concatenate((np.zeros((num_samples // 2)), np.ones((num_samples // 2))))
print(X.shape, y.shape)

(20, 300, 7) (20,)


In [4]:
scaler = ThreeDStandardScaler()
X_scaled = scaler.fit_transform(X, y)
print(X_scaled.shape)

(20, 300, 7)


In [5]:
from collections import namedtuple

Input = namedtuple("Input", "X y")
input = Input(X_scaled, y)

In [6]:
def test_model(model, input: Input = input):
    batch_size = 2
    epochs = 2
    X = tf.convert_to_tensor(input.X, dtype=tf.float64)
    y = tf.convert_to_tensor(input.y, dtype=tf.int8)

    lrate = keras.callbacks.LearningRateScheduler(step_decay)
    callbacks_list = [lrate]

    model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)
    score = model.evaluate(X_scaled, y)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

In [7]:
test_model(model=build_dummy_tf_classifier(input_shape=X.shape))

Activation was set to None which is not supported.
Epoch 1/2
9/9 [==============================] - 0s 16ms/step - loss: 6.1416 - accuracy: 0.5556 - val_loss: 15.4249 - val_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 0s 65ms/step - loss: 7.0699 - accuracy: 0.5000
Test loss: 7.0698957443237305
Test accuracy: 0.5


In [8]:
test_model(model=build_dense_model(input_shape=X.shape))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 2100)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                134464    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 134,529
Trainable params: 134,529
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
9/9 [==============================] - 0s 11ms/step - loss: 1.7747 - accuracy: 0.4444 - val_loss: 1.1392 - val_accuracy: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 65ms/step - loss: 0.1734 - accuracy: 0.5000
Test loss: 0.17339317500591278
Test accuracy: 0.5


In [9]:
test_model(model=build_cnn_model(input_shape=X.shape, pooling="max"))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 300, 7)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 300, 32)           1152      
_________________________________________________________________
batch_normalization (BatchNo (None, 300, 32)           128       
_________________________________________________________________
re_lu (ReLU)                 (None, 300, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 300, 32)           5152      
_________________________________________________________________
batch_normalization_1 (Batch (None, 300, 32)           128       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 300, 32)           0   

In [10]:
test_model(model=build_lstm_model(input_shape=X.shape))

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 300, 7)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 300, 128)          69632     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
flatten_2 (Flatten)          (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 201,345
Trainable params: 201,345
Non-trainable params: 0
_____________________________________________________

use real data